In [1]:
%load_ext autoreload
%autoreload 2
import warnings
import numpy as np
import time
import os
from dotenv import load_dotenv
os.environ["USE_PYGEOS"] = "0"
import napari_sparrow as nas
from spatialdata import read_zarr
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
%run own_and_modified_functions.ipynb
warnings.simplefilter(action='ignore')

the value of the environment variable BASIC_DCT_BACKEND is not in ["JAX","SCIPY"]


In [2]:
OUTPUT_DIR = '/home/wout/Documents/Thesis_lokaal/Mouse_Liver_Resolve_Data'
sdata = read_zarr( os.path.join(  OUTPUT_DIR, 'sdata.zarr'  ) )
anndata = sc.read('/home/wout/Documents/Thesis_lokaal/Mouse_Liver_Resolve_Data/anndataNucA11') # reset original anndata
# delete anndata
del sdata.table
# create anndata
sdata.table = anndata

In [3]:
n_neighbors = 15
n_PCAs = 23
cluster_resolution = 1
# make umap and do leiden clustering with scanpy functions
make_umap(sdata,n_neighbors=n_neighbors,n_PCAs=n_PCAs)
column_name = 'leiden_'+str(n_PCAs)+'_'+str(n_neighbors)+'_'+str(cluster_resolution)
sc.tl.leiden(sdata.table,resolution=cluster_resolution,key_added=column_name)

In [4]:
path_mg = "/home/wout/Documents/Thesis_lokaal/Mouse_Liver_Resolve_Data/markerGeneListBasic.csv"
scores = make_umap_and_perform_leiden_annotation(sdata,path_mg,n_PCAs,n_neighbors,cluster_resolution,norm_expr_var=True,min_score='Quantile',min_score_q=25,scale_score='Robust',scale_score_q=1)


0.25


In [5]:
scores


,Hepa,LSEC,Endo vein,Chol,Kupf,Stel,Fibr,B cell,Meso
0,0.295484,0.482843,0.113675,0.047148,0.078107,0.111511,0.046796,0.003552,NaN
1,0.550029,0.003252,NaN,0.119755,NaN,NaN,NaN,0.000629,0.030956
2,0.904219,0.003528,0.008788,0.211904,0.024651,0.086014,0.062950,0.001388,0.042930
3,1.014885,NaN,0.009428,0.244556,0.042210,0.184855,0.135827,NaN,0.015170
4,0.902658,NaN,0.007077,0.208165,0.032742,0.093093,0.057237,0.000637,NaN
5,0.216939,0.111578,0.015578,NaN,1.181335,0.106381,0.062103,0.009039,0.016458
6,0.242958,0.035244,0.050244,NaN,0.115096,1.084494,0.631016,0.001669,0.059213
7,0.859246,NaN,0.007322,0.224020,0.027180,0.227757,0.082646,NaN,0.043615
8,0.648057,NaN,NaN,0.168699,NaN,0.141526,0.103666,NaN,0.052964
9,0.782869,0.004790,NaN,0.176393,NaN,0.096773,0.068014,NaN,0.048627


In [12]:
scores_draft = scores.copy(deep=True)
max_scores = scores.max(axis=1)
second_scores = scores.apply(lambda x: x.nlargest(2).values[-1], axis=1)
cleanl_per_cluster = (max_scores - second_scores) / ((max_scores + second_scores) / 2)
cleanl_per_cluster


0     0.481441
1     1.284815
2     1.240571
3     1.223287
4     1.250412
5     1.379409
6     0.528679
7     1.161891
8     1.173809
9     1.264466
10    1.240071
11    0.034216
12    1.504040
13    0.216256
14    1.432106
15    0.841217
16    0.309521
17    1.737308
18    1.191247
19    0.024436
20    0.778043
21    0.327492
22    0.623045
dtype: float64

In [13]:
third_scores = scores.apply(lambda x: x.nlargest(3).values[-1], axis=1)
cleanl_per_cluster_extra = (max_scores - third_scores) / ((max_scores + third_scores) / 2)
cleanl_per_cluster_extra


0     1.237741
1     1.786870
2     1.652552
3     1.383684
4     1.626041
5     1.654800
6     1.267897
7     1.172799
8     1.283034
9     1.559945
10    1.688521
11    0.479750
12    1.680637
13    0.223517
14    1.949029
15    1.352998
16    1.196491
17    1.788877
18         NaN
19    0.938449
20    1.226456
21    0.530680
22    0.864204
dtype: float64

In [14]:
sc_leiden_cellt = scores.idxmax(axis=1).to_dict()
sc_leiden_cellt

{0: 'LSEC',
 1: 'Hepa',
 2: 'Hepa',
 3: 'Hepa',
 4: 'Hepa',
 5: 'Kupf',
 6: 'Stel',
 7: 'Hepa',
 8: 'Hepa',
 9: 'Hepa',
 10: 'Hepa',
 11: 'Meso',
 12: 'Hepa',
 13: 'Kupf',
 14: 'Hepa',
 15: 'Chol',
 16: 'Endo vein',
 17: 'B cell',
 18: 'Hepa',
 19: 'LSEC',
 20: 'LSEC',
 21: 'Fibr',
 22: 'Kupf'}

In [15]:
for i in range(23):
    if cleanl_per_cluster[i] < 0.5:
        scores_draft.loc[i].at[scores_draft.idxmax(axis=1)[i]] = np.nan 
        sc_leiden_cellt[i] = sc_leiden_cellt[i] + '/' + scores_draft.idxmax(axis=1)[i]
        if cleanl_per_cluster_extra[i] < 0.5:
            scores_draft.loc[i].at[scores_draft.idxmax(axis=1)[i]] = np.nan 
            sc_leiden_cellt[i] = sc_leiden_cellt[i] + '/' + scores_draft.idxmax(axis=1)[i]
            # calculate proportions of max_scores[i], second_scores[i] and third_scores[i]
            sum = max_scores[i] + second_scores[i] + third_scores[i]
            p1 = round(100*max_scores[i]/sum)
            p2 = round(100*second_scores[i]/sum)
            p3 = round(100*third_scores[i]/sum)
            sc_leiden_cellt[i] = sc_leiden_cellt[i] + '(' + str(p1) + '%/' + str(p2) + '%/' + str(p3) + '%)'
        else:
            sum = max_scores[i] + second_scores[i]
            p1 = round(100*max_scores[i]/sum)
            p2 = round(100*second_scores[i]/sum)
            sc_leiden_cellt[i] = sc_leiden_cellt[i] + '(' + str(p1) + '%/' + str(p2) + '%)'
sc_leiden_cellt


{0: 'LSEC/Hepa(62%/38%)',
 1: 'Hepa',
 2: 'Hepa',
 3: 'Hepa',
 4: 'Hepa',
 5: 'Kupf',
 6: 'Stel',
 7: 'Hepa',
 8: 'Hepa',
 9: 'Hepa',
 10: 'Hepa',
 11: 'Meso/Fibr/Stel(39%/37%/24%)',
 12: 'Hepa',
 13: 'Kupf/Meso/B cell(38%/31%/31%)',
 14: 'Hepa',
 15: 'Chol',
 16: 'Endo vein/LSEC(58%/42%)',
 17: 'B cell',
 18: 'Hepa',
 19: 'LSEC/Endo vein(51%/49%)',
 20: 'LSEC',
 21: 'Fibr/B cell(58%/42%)',
 22: 'Kupf'}